In [2]:
import pandas as pd
import requests
import json
import csv
import time
import datetime

test API hits to see numbers and make sure queries are working

In [3]:
test_query = ''
test_sub = 'healthyfood'
test_after = '1564923895' # aug 1 2019
test_before = '1565228695'


url = ('https://api.pushshift.io/reddit/search/submission/?title='+str(test_query)+'&size=10000&after='
       +str(test_after)+'&before='+str(test_before)+'&subreddit='+str(test_sub))
print(url)
r = requests.get(url)
data = json.loads(r.text)

https://api.pushshift.io/reddit/search/submission/?title=&size=10000&after=1564923895&before=1565228695&subreddit=healthyfood


In [4]:
len(list(data.values())[0])


82

Define functions for querying the pushshift API


In [5]:
def getPushshiftData(query, after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=10000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

In [6]:
def collectSubData(subm):
    subData = list() #list to store data points
    title = subm['title']
    url = subm['url']
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"    
    try:
        txt = subm['selftext']
    except KeyError:
        txt = "NaN"   
    author = subm['author']
    sub_id = subm['id']
    score = subm['score']
    created = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    numComms = subm['num_comments']
    permalink = subm['permalink']
    
    subData.append((sub_id,title,txt,url,author,score,created,numComms,permalink,flair))
    subStats[sub_id] = subData

Parameters; you have to convert dates to timestamps which you can do on any number of websites


In [7]:
#list of post IDs
post_ids = []

#Subreddit to query
sub="healthyfood"

# Unix timestamp of date to crawl from.
# 1/1/2016
after = "1136082436"  # 1/1/2006

# timestamp of date to crawl to
before = "1564779482"   # 8/2/2019
query = ""

subCount = 0
subStats = {}


Can only pull down 1000 posts at a time, so we call getPushiftData on the end of the last post pulled until there are no posts left


In [8]:
data = getPushshiftData(query, after, before, sub)

while len(data) > 0:
    for submission in data:
        collectSubData(submission)
        subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
    print(len(data))
    #print(submission['title'])
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = getPushshiftData(query, after, before, sub)
    time.sleep(5)

https://api.pushshift.io/reddit/search/submission/?title=&size=10000&after=1136082436&before=1564779482&subreddit=healthyfood
1000
2013-08-07 16:07:35
https://api.pushshift.io/reddit/search/submission/?title=&size=10000&after=1375906055&before=1564779482&subreddit=healthyfood


KeyboardInterrupt: 

In [9]:
# number of posts in your data
len(subStats)

1000

Writes CSV with output

In [10]:
file = 'reddit_'+ sub + '_submissions.csv'
with open(file, 'w', newline='', encoding='utf-8') as file: 
        a = csv.writer(file, delimiter=',')
        headers = ["post_id","title","text","url","author","score","created_date","numcomms","permalink","flair"]
        a.writerow(headers)
        upload_count = 0
        for sub in subStats:
            a.writerow(subStats[sub][0])
            upload_count+=1
            